# OpenAI Tool calling capabilities with Unity Catalog

## Prerequisites

**API Key**
To run this tutorial, you will need an OpenAI API key. 

Once you have acquired your key, set it to the environment variable `OPENAI_API_KEY`.

Below, we validate that this key is set properly in your environment.

**Packages**

To interface with both UnityCatalog and OpenAI, you will need to install the following packages:

```shell
pip install openai unitycatalog-openai
```

In [1]:
import os

assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

## Configuration and Client setup

In order to connect to your Unity Catalog server, you'll need an instance of the `ApiClient` from the `unitycatalog-client` package. 

> Note: If you don't already have a Catalog and a Schema created, be sure to create them before running this notebook and adjust the `CATALOG` and `SCHEMA` variables below to suit.

In [ ]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.ai.openai.toolkit import UCFunctionToolkit
from unitycatalog.client import ApiClient, Configuration

In [3]:
config = Configuration()
config.host = "http://localhost:8080/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

CATALOG = "AICatalog"
SCHEMA = "AISchema"

## Define functions and register them to Unity Catalog

In this next section, we'll be defining a Python function and creating it within Unity Catalog so that it can be retrieved and used as a tool with our calls to OpenAI. 

There are a few things to keep in mind when creating functions for use with the `create_python_function` API:

- Ensure that your have properly defined types for all arguments and for the return of the function.
- Ensure that you have a Google-style docstring defined that includes descriptions for the function, each argument, and the return of the function. This is critical, as these are used to populate the metadata associated with the function within Unity Catalog, providing contextual data for an LLM to understand when and how to call the tool associated with this function.
- If there are packages being called that are not part of core Python, ensure that the import statements are locally scoped (defined within the function body).

In [4]:
def impact_energy(mass: float, velocity: float, composition: str) -> float:
    """
    Calculates the amount of energy transmitted to Earth by an interstellar body upon impact.

    Args:
        mass (float): Mass of the body in kilograms.
        velocity (float): Velocity of the body in meters per second.
        composition (str): Elemental composition of the body (e.g., 'iron', 'stone', 'ice', 'diamond', 'uranium', 'gold', 'titanium', 'lithium').

    Returns:
        Energy transmitted to Earth in joules.
    """
    if mass <= 0:
        raise ValueError("Mass must be positive.")
    if velocity <= 0:
        raise ValueError("Velocity must be positive.")

    energy_transmission = {
        "iron": 0.9,  # High survival rate through atmosphere
        "stone": 0.5,  # Moderate survival rate
        "ice": 0.1,  # Low survival rate due to ablation
        "diamond": 0.8,  # High melting point and thermal conductivity
        "uranium": 0.85,  # High density and melting point
        "gold": 0.7,  # High density but lower melting point
        "titanium": 0.75,  # High strength and melting point
        "lithium": 0.2,  # Low density and melting point
    }

    transmission_coefficient = energy_transmission.get(composition.lower())

    if transmission_coefficient is None:
        raise ValueError(
            f"Unknown composition: {composition}. Must be one of: {list(energy_transmission.keys())}"
        )

    kinetic_energy = 0.5 * mass * velocity**2

    energy_transmitted = kinetic_energy * transmission_coefficient

    return energy_transmitted

In [5]:
client.create_python_function(func=impact_energy, catalog=CATALOG, schema=SCHEMA, replace=True)

FunctionInfo(name='impact_energy', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='mass', type_text='DOUBLE', type_json='{"name": "mass", "type": "double", "nullable": false, "metadata": {"comment": "Mass of the body in kilograms."}}', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='Mass of the body in kilograms.'), FunctionParameterInfo(name='velocity', type_text='DOUBLE', type_json='{"name": "velocity", "type": "double", "nullable": false, "metadata": {"comment": "Velocity of the body in meters per second."}}', type_name=<ColumnTypeName.DOUBLE: 'DOUBLE'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='Velocity of the body in meters per second.'), FunctionParamet

## Build a tool calling Agent with OpenAI

In order to let a GenAI service like a GPT model hosted by OpenAI use our functions, we need to register them as tools. 

To do this, we'll import the Unity Catalog AI OpenAI integration package and utilize the `UCFunctionToolkit` class to construct the interface we need to register tools. 

In [6]:
toolkit = UCFunctionToolkit(function_names=[f"{CATALOG}.{SCHEMA}.impact_energy"], client=client)
tools = toolkit.tools
tools

[{'type': 'function',
  'function': {'name': 'AICatalog__AISchema__impact_energy',
   'strict': True,
   'parameters': {'properties': {'mass': {'description': 'Mass of the body in kilograms.',
      'title': 'Mass',
      'type': 'number'},
     'velocity': {'description': 'Velocity of the body in meters per second.',
      'title': 'Velocity',
      'type': 'number'},
     'composition': {'description': 'Elemental composition of the body (e.g., "iron", "stone", "ice", "diamond", "uranium", "gold", "titanium", "lithium").',
      'title': 'Composition',
      'type': 'string'}},
    'title': 'AICatalog__AISchema__impact_energy__params',
    'type': 'object',
    'additionalProperties': False,
    'required': ['mass', 'velocity', 'composition']},
   'description': 'Calculates the amount of energy transmitted to Earth by an interstellar body upon impact.'}}]

## Interface with OpenAI's GPT model with tool calling capabilities

In the following code block, we submit our messages to OpenAI. As with a standard query, we provide both a system prompt message and a user message. 

In order to allow the LLM to utilize tool calling capabilities, we pass our toolkit definitions (`toolkit.tools`) to the `tools` argument within the OpenAI SDK `chat.completions.create` API. With the tool definitions provided, OpenAI's LLM is available to contextually 'decide' when it is appropriate to request a tool call to be executed with its response. 

In the case of the question that we're providing, OpenAI will recognize that it should call both of our tools in order to facilitate an accurate answer.

In [7]:
import openai

initial_messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that provides realistic, verbose, and hightly technical responses to abstract 'what if?' "
        "questions that curious users ask. When responding to a question, utilize tools that are available to you to provide the most"
        "accurate answers possible. After getting a response to a tool call, use that to explain in full detail the contextual "
        "information to provide a full accounting of what the real-world effects would be for the question being posed and include "
        "a thorough explanation that covers the topic in as much detail as you can.",
    },
    {
        "role": "user",
        "content": "What would happen if a sphere of iron with a mass of 64008.5kg struck the Earth traveling at 773542.99 m/s?",
    },
]

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=initial_messages,
    tools=tools,
)

response

ChatCompletion(id='chatcmpl-AY2Q4daxucRRao7I4QFpqHSSqt4dW', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_R9sXaB0d3oISGogH9awqCpFw', function=Function(arguments='{"mass":64008.5,"velocity":773542.99,"composition":"iron"}', name='AICatalog__AISchema__impact_energy'), type='function')]))], created=1732675816, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0705bf87c0', usage=CompletionUsage(completion_tokens=35, prompt_tokens=281, total_tokens=316, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [8]:
from unitycatalog.ai.openai.utils import generate_tool_call_messages

messages = generate_tool_call_messages(response=response, client=client)

messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_R9sXaB0d3oISGogH9awqCpFw',
    'function': {'arguments': '{"mass":64008.5,"velocity":773542.99,"composition":"iron"}',
     'name': 'AICatalog__AISchema__impact_energy'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '{"content": "1.7235308972987406e+16"}',
  'tool_call_id': 'call_R9sXaB0d3oISGogH9awqCpFw'}]

In [ ]:
# Prepend the message history
final_messages = initial_messages + messages

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=final_messages,
    tools=tools,
)

response.choices[0].message.content

To understand the implications of a sphere of iron with a mass of 64,008.5 kg striking the Earth at a velocity of 773,542.99 m/s, we first need to calculate the energy released upon impact and then examine the potential consequences of such an event.

### Energy Release Calculation
The energy transmitted to Earth by this impact can be calculated using the formula for kinetic energy, which is given by:

\[
E = \frac{1}{2} m v^2
\]

Where:
- \(E\) is the kinetic energy,
- \(m\) is the mass of the object (64,008.5 kg),
- \(v\) is the velocity of the object (773,542.99 m/s).

The calculation yields an impact energy of approximately \(1.72 \times 10^{16}\) joules. To put this in perspective, this amount of energy is equivalent to about:

- 4.1 Megatons of TNT.
- Over four times the energy released by the atomic bomb dropped on Hiroshima, which was about 15 kilotons.

### Potential Consequences of the Impact

1. **Impact Location and Depth**: The exact effects would depend significantly on w